In [4]:
import pandas as pd
import numpy as np
import math
import arch
import scipy

import seaborn as sns
import matplotlib.pyplot as plt
from arch import arch_model
import scipy.stats as stats

In [14]:
backtest_fhs = pd.read_csv('backtest_fhs.csv')

backtest_hs = pd.read_csv('backtest_hs.csv')

backtest_garch = pd.read_csv('backtest_garch.csv')

In [6]:
backtest_fhs['Indicator 99%'] = backtest_fhs['daily linearised loss'].gt(backtest_fhs['VaR FHS 99%'])
backtest_fhs['Indicator 95%'] = backtest_fhs['daily linearised loss'].gt(backtest_fhs['VaR FHS 95%'])
T = len(backtest_fhs)
H_99 = backtest_fhs['Indicator 99%'].sum()
H_95 = backtest_fhs['Indicator 95%'].sum()

pi_99 = H_99/len(backtest_fhs)
pi_95 = H_95/len(backtest_fhs)

alpha_99 = 0.99
alpha_95 = 0.95

In [9]:
def L(x,T,H):
    return(np.power((1-x),T-H)*np.power(x,H))    
def LRuc(alpha,pi_,T,H):
        d = -2*np.log(L(1-alpha,T,H)/L(pi_,T,H))
        return d

In [10]:
def cover_test( alpha_99,alpha_95,pi_99,pi_95,T,H_99,H_95):
    L_99 = LRuc(alpha_99,pi_99,T,H_99)
    L_95 = LRuc(alpha_95,pi_95,T,H_95)
    
    p_95 = 1 - stats.chi2.cdf(L_95,1)
    p_99 = 1 - stats.chi2.cdf(L_99,1)
    
    return([p_95,L_95,p_99,L_99])

In [12]:
from tabulate import tabulate
result = cover_test(0.99,0.95,pi_99,pi_95,T,H_99,H_95)
t = tabulate([['0.99',result[3],result[2]],['0.95',result[1],result[0]]], headers = ['Alpha','LRuc', 'p-value'])
print(t)

  Alpha      LRuc    p-value
-------  --------  ---------
   0.99  0.19809    0.656267
   0.95  0.131711   0.716665


In [15]:
backtest_hs

,Date,SSO,daily linearised loss,VaR 99,VaR 95,VaR 99%,VaR 95%,VaR Violations 99,VaR Violations 95,ES 95%,ES 99%
0,2008-11-10,14.1800,0.042795,0.100203,0.054405,0.100203,0.054405,5.0,25.0,0.085594,0.141279
1,2008-11-11,13.7450,0.031157,0.100203,0.054405,0.100203,0.054405,5.0,25.0,0.085594,0.141279
2,2008-11-12,12.4200,0.101367,0.101407,0.054481,0.101407,0.054481,5.0,24.0,0.086891,0.141279
3,2008-11-13,14.0500,-0.123314,0.101407,0.054481,0.101407,0.054481,5.0,25.0,0.087470,0.141279
4,2008-11-14,12.7150,0.099840,0.101407,0.054841,0.101407,0.054841,5.0,24.0,0.088834,0.141279
5,2008-11-17,12.2250,0.039299,0.101407,0.054841,0.101407,0.054841,5.0,25.0,0.089274,0.141279
6,2008-11-18,12.3150,-0.007335,0.101407,0.054841,0.101407,0.054841,5.0,25.0,0.089274,0.141279
7,2008-11-19,11.0900,0.104774,0.104780,0.057090,0.104780,0.057090,5.0,24.0,0.090617,0.141279
8,2008-11-20,9.5050,0.154226,0.105444,0.057823,0.105444,0.057823,4.0,24.0,0.092574,0.150259
9,2008-11-21,10.6050,-0.109508,0.105444,0.057823,0.105444,0.057823,5.0,25.0,0.095040,0.151052


In [17]:
backtest_hs['Indicator 99%'] = backtest_hs['daily linearised loss'].gt(backtest_hs['VaR 99%'])
backtest_hs['Indicator 95%'] = backtest_hs['daily linearised loss'].gt(backtest_hs['VaR 95%'])
T = len(backtest_hs)
H_99 = backtest_hs['Indicator 99%'].sum()
H_95 = backtest_hs['Indicator 95%'].sum()

pi_99 = H_99/len(backtest_hs)
pi_95 = H_95/len(backtest_hs)

alpha_99 = 0.99
alpha_95 = 0.95

In [18]:
def cover_test( alpha_99,alpha_95,pi_99,pi_95,T,H_99,H_95):
    L_99 = LRuc(alpha_99,pi_99,T,H_99)
    L_95 = LRuc(alpha_95,pi_95,T,H_95)
    
    p_95 = 1 - stats.chi2.cdf(L_95,1)
    p_99 = 1 - stats.chi2.cdf(L_99,1)
    
    return([p_95,L_95,p_99,L_99])

In [20]:
backtest_garch

,Date,SSO,daily linearised loss,VaR GARCH 99%,VaR GARCH 95%,ES GARCH 99%,ES GARCH 95%
0,2008-11-10,14.1800,0.042795,0.223624,0.142401,0.285722,0.191887
1,2008-11-11,13.7450,0.031157,0.213226,0.135698,0.272628,0.183269
2,2008-11-12,12.4200,0.101367,0.201599,0.128534,0.258139,0.173473
3,2008-11-13,14.0500,-0.123314,0.210669,0.133996,0.269076,0.181196
4,2008-11-14,12.7150,0.099840,0.227551,0.145052,0.290048,0.195666
5,2008-11-17,12.2250,0.039299,0.231304,0.147213,0.294291,0.198810
6,2008-11-18,12.3150,-0.007335,0.220239,0.140193,0.280109,0.189208
7,2008-11-19,11.0900,0.104774,0.206947,0.131727,0.263207,0.177755
8,2008-11-20,9.5050,0.154226,0.215707,0.137411,0.273925,0.185231
9,2008-11-21,10.6050,-0.109508,0.245641,0.157030,0.311951,0.211673


In [19]:
result = cover_test(0.99,0.95,pi_99,pi_95,T,H_99,H_95)
t = tabulate([['0.99',result[3],result[2]],['0.95',result[1],result[0]]], headers = ['Alpha','LRuc', 'p-value'])
print(t)

  Alpha      LRuc    p-value
-------  --------  ---------
   0.99  0.426275  0.513823
   0.95  3.15565   0.0756647


In [23]:
backtest_garch['Indicator 99%'] = backtest_garch['daily linearised loss'].gt(backtest_garch['VaR GARCH 99%'])
backtest_garch['Indicator 95%'] = backtest_garch['daily linearised loss'].gt(backtest_garch['VaR GARCH 95%'])
T = len(backtest_garch)
H_99 = backtest_garch['Indicator 99%'].sum()
H_95 = backtest_garch['Indicator 95%'].sum()

pi_99 = H_99/len(backtest_garch)
pi_95 = H_95/len(backtest_garch)

alpha_99 = 0.99
alpha_95 = 0.95

In [24]:
result = cover_test(0.99,0.95,pi_99,pi_95,T,H_99,H_95)
t = tabulate([['0.99',result[3],result[2]],['0.95',result[1],result[0]]], headers = ['Alpha','LRuc', 'p-value'])
print(t)

  Alpha      LRuc    p-value
-------  --------  ---------
   0.99  0.426275  0.513823
   0.95  3.50732   0.0610981


In [78]:

"from https://github.com/BayerSe/VaR-Backtesting/blob/master/backtest.py"

class Backtest:
    def __init__(self, actual, forecast, alpha):
        self.index = actual.index
        self.actual = actual.values
        self.forecast = forecast.values
        self.alpha = alpha

    def hit_series(self):
        return (self.actual < self.forecast) * 1

    def number_of_hits(self):
        return self.hit_series().sum()

    def hit_rate(self):
        return self.hit_series().mean()

    def expected_hits(self):
        return self.actual.size * self.alpha

    def duration_series(self):
        hit_series = self.hit_series()
        hit_series[0] = 1
        hit_series[-1] = 1
        return np.diff(np.where(hit_series == 1))[0]

    def plot(self, file_name=None):

        # Re-add the time series index
        r = pd.Series(self.actual, index=self.index)
        q = pd.Series(self.forecast, index=self.index)

        sns.set_context("paper")
        sns.set_style("whitegrid", {"font.family": "serif", "font.serif": "Computer Modern Roman", "text.usetex": True})

        # Hits
        ax = r[r <= q].plot(color="red", marker="o", ls="None", figsize=(6, 3.5))
        for h in r[r <= q].index:
            plt.axvline(h, color="black", alpha=0.4, linewidth=1, zorder=0)

        # Positive returns
        r[q < r].plot(ax=ax, color="green", marker="o", ls="None")

        # Negative returns but no hit
        r[(q <= r) & (r <= 0)].plot(ax=ax, color="orange", marker="o", ls="None")

        # VaR
        q.plot(ax=ax, grid=False, color="black", rot=0)

        # Axes
        plt.xlabel("")
        plt.ylabel("Log Return")
        ax.yaxis.grid()

        sns.despine()
        if file_name is None:
            plt.show()
        else:
            plt.savefig(file_name, bbox_inches="tight")
        plt.close("all")

    def tick_loss(self, return_mean=True):
        loss = (self.alpha - self.hit_series()) * (self.actual - self.forecast)
        if return_mean:
            return loss.mean()
        else:
            return loss

    def smooth_loss(self, delta=25, return_mean=True):
        """Gonzalez-Rivera, Lee and Mishra (2004)"""
        loss = ((self.alpha - (1 + np.exp(delta*(self.actual - self.forecast)))**-1) * (self.actual - self.forecast))
        if return_mean:
            return loss.mean()
        else:
            return loss

    def quadratic_loss(self, return_mean=True):
        """Lopez (1999); Martens et al. (2009)"""
        loss = (self.hit_series() * (1 + (self.actual - self.forecast)**2))
        if return_mean:
            return loss.mean()
        else:
            return loss

    def firm_loss(self, c=1, return_mean=True):
        """Sarma et al. (2003)"""
        loss = (self.hit_series() * (1 + (self.actual - self.forecast)**2) - c*(1-self.hit_series()) * self.forecast)
        if return_mean:
            return loss.mean()
        else:
            return loss

    def lr_bt(self):
        """Likelihood ratio framework of Christoffersen (1998)"""
        hits = self.hit_series()   # Hit series
        tr = hits[1:] - hits[:-1]  # Sequence to find transitions

        # Transitions: nij denotes state i is followed by state j nij times
        n01, n10 = (tr == 1).sum(), (tr == -1).sum()
        n11, n00 = (hits[1:][tr == 0] == 1).sum(), (hits[1:][tr == 0] == 0).sum()

        # Times in the states
        n0, n1 = n01 + n00, n10 + n11
        n = n0 + n1

        # Probabilities of the transitions from one state to another
        p01, p11 = n01 / (n00 + n01), n11 / (n11 + n10)
        p = n1 / n

        if n1 > 0:
            # Unconditional Coverage
            uc_h0 = n0 * np.log(1 - self.alpha) + n1 * np.log(self.alpha)
            uc_h1 = n0 * np.log(1 - p) + n1 * np.log(p)
            uc = -2 * (uc_h0 - uc_h1)

            # Independence
            ind_h0 = (n00 + n01) * np.log(1 - p) + (n01 + n11) * np.log(p)
            ind_h1 = n00 * np.log(1 - p01) + n01 * np.log(p01) + n10 * np.log(1 - p11)
            if p11 > 0:
                ind_h1 += n11 * np.log(p11)
            ind = -2 * (ind_h0 - ind_h1)

            # Conditional coverage
            cc = uc + ind

            # Stack results
            df = pd.concat([pd.Series([uc, ind, cc]),
                            pd.Series([1 - stats.chi2.cdf(uc, 1),
                                       1 - stats.chi2.cdf(ind, 1),
                                       1 - stats.chi2.cdf(cc, 2)])], axis=1)
        else:
            df = pd.DataFrame(np.zeros((3, 2))).replace(0, np.nan)

        # Assign names
        df.columns = ["Statistic", "p-value"]
        df.index = ["Unconditional", "Independence", "Conditional"]

        return df
    


In [53]:
forecast_fhs = backtest_fhs['VaR FHS 99%'].copy()
actual_fhs = backtest_fhs['daily linearised loss'].copy()
bt = Backtest(actual=actual_fhs, forecast = forecast_fhs, alpha=0.99)
bt.lr_bt()

,Statistic,p-value
Unconditional,0.196338,0.657693
Independence,5.723809,0.016736
Conditional,5.920147,0.051815


In [50]:
forecast_fhs = backtest_fhs['VaR FHS 95%'].copy()
actual_fhs = backtest_fhs['daily linearised loss'].copy()
bt = Backtest(actual=actual_fhs, forecast = forecast_fhs, alpha=0.95)
bt.lr_bt()

,Statistic,p-value
Unconditional,0.128432,0.720063
Independence,0.001472,0.969395
Conditional,0.129904,0.718531


In [51]:
forecast_garch = backtest_garch['VaR GARCH 99%'].copy()
actual_garch = backtest_garch['daily linearised loss'].copy()
bt = Backtest(actual=actual_garch, forecast = forecast_garch, alpha=0.99)
bt.lr_bt()

,Statistic,p-value
Unconditional,0.423722,0.515086
Independence,6.063243,0.013802
Conditional,6.486964,0.010867


In [41]:
forecast_garch = backtest_garch['VaR GARCH 95%'].copy()
actual_garch = backtest_garch['daily linearised loss'].copy()
bt = Backtest(actual=actual_garch, forecast = forecast_garch, alpha=0.95)
bt.lr_bt()

,Statistic,p-value
Unconditional,3.490736,0.061713
Independence,0.053930,0.816361
Conditional,3.544666,0.169936


In [46]:
forecast_hs = backtest_hs['VaR 99%'].copy()
actual_hs = backtest_hs['daily linearised loss'].copy()
bt = Backtest(actual=actual_hs, forecast = forecast_hs, alpha=0.99)
bt.lr_bt()

,Statistic,p-value
Unconditional,0.423722,0.515086
Independence,11.630311,0.000649
Conditional,12.054033,0.002413


In [45]:
forecast_hs = backtest_hs['VaR 95%'].copy()
actual_hs = backtest_hs['daily linearised loss'].copy()
bt = Backtest(actual=actual_hs, forecast = forecast_hs, alpha=0.95)
bt.lr_bt()

,Statistic,p-value
Unconditional,3.139888,0.076399
Independence,14.422454,0.000146
Conditional,17.562342,0.000154


In [54]:
backtest_fhs

,Date,SSO,daily linearised loss,EWMA Volatility,EWMA Standardized Residuals,VaR FHS 95%,VaR FHS 99%,ES FHS 95%,ES FHS 99%
0,2008-11-10,14.1800,0.042795,0.014555,2.940255,0.176873,0.268064,0.252398,0.436226
1,2008-11-11,13.7450,0.031157,0.013920,2.238268,0.169163,0.256379,0.241397,0.417212
2,2008-11-12,12.4200,0.101367,0.013218,7.668883,0.160628,0.243444,0.229217,0.396161
3,2008-11-13,14.0500,-0.123314,0.013834,-8.914108,0.168109,0.254783,0.239893,0.414613
4,2008-11-14,12.7150,0.099840,0.014961,6.673274,0.181812,0.275550,0.259446,0.448408
5,2008-11-17,12.2250,0.039299,0.015286,2.570910,0.185761,0.281535,0.265082,0.458148
6,2008-11-18,12.3150,-0.007335,0.014561,-0.503736,0.176951,0.268183,0.252510,0.436419
7,2008-11-19,11.0900,0.104774,0.013695,7.650777,0.166420,0.252222,0.237482,0.410446
8,2008-11-20,9.5050,0.154226,0.014326,10.765526,0.174092,0.263849,0.248430,0.429368
9,2008-11-21,10.6050,-0.109508,0.016339,-6.702126,0.198558,0.300930,0.283344,0.489710


In [57]:
backtest_fhs['Indicator 99%'] = backtest_fhs['daily linearised loss'].gt(backtest_fhs['VaR FHS 99%'])
backtest_fhs['Indicator 95%'] = backtest_fhs['daily linearised loss'].gt(backtest_fhs['VaR FHS 95%'])


,Date,SSO,daily linearised loss,EWMA Volatility,EWMA Standardized Residuals,VaR FHS 95%,VaR FHS 99%,ES FHS 95%,ES FHS 99%,Indicator 99%,Indicator 95%
0,2008-11-10,14.1800,0.042795,0.014555,2.940255,0.176873,0.268064,0.252398,0.436226,False,False
1,2008-11-11,13.7450,0.031157,0.013920,2.238268,0.169163,0.256379,0.241397,0.417212,False,False
2,2008-11-12,12.4200,0.101367,0.013218,7.668883,0.160628,0.243444,0.229217,0.396161,False,False
3,2008-11-13,14.0500,-0.123314,0.013834,-8.914108,0.168109,0.254783,0.239893,0.414613,False,False
4,2008-11-14,12.7150,0.099840,0.014961,6.673274,0.181812,0.275550,0.259446,0.448408,False,False
5,2008-11-17,12.2250,0.039299,0.015286,2.570910,0.185761,0.281535,0.265082,0.458148,False,False
6,2008-11-18,12.3150,-0.007335,0.014561,-0.503736,0.176951,0.268183,0.252510,0.436419,False,False
7,2008-11-19,11.0900,0.104774,0.013695,7.650777,0.166420,0.252222,0.237482,0.410446,False,False
8,2008-11-20,9.5050,0.154226,0.014326,10.765526,0.174092,0.263849,0.248430,0.429368,False,False
9,2008-11-21,10.6050,-0.109508,0.016339,-6.702126,0.198558,0.300930,0.283344,0.489710,False,False


In [71]:
backtest_fhs['Zeta 99%'] = backtest_fhs['Indicator 99%']*(backtest_fhs['daily linearised loss'] - backtest_fhs['ES FHS 99%'])
backtest_fhs['Zeta² 99%'] = backtest_fhs['Zeta 99%']*2
Z = backtest_fhs['Zeta 99%'].sum()/ np.sqrt(backtest_fhs['Zeta² 99%'].sum())
p



0.05524912088761929

In [79]:
bz = Backtest(actual = backtest_fhs['daily linearised loss'], forecast = backtest_fhs['ES FHS 99%'], alpha = 0.99)
bz.test_ES()

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"

In [143]:
class Backtest_2:
    def __init__(self, actual, forecast, alpha):
        #self.index = actual.index
        self.actual = actual
        self.forecast = forecast
        self.alpha = alpha
        
    def test_ES(self):
        df = pd.DataFrame(self.actual)
        df.rename(columns={df.columns[0]:'actual'}, inplace=True)
        df['forecast'] = self.forecast
        str_ = 'Indicator ' + str(int(self.alpha*100)) + '%'
        df[str_] = df['actual'].gt(df['forecast'])
        z_ = 'Zeta' + str(int(self.alpha*100)) + '%'
        z2 ='Zeta2 ' + str(int(self.alpha*100)) + '%'
        df[z_] = df[str_]*(df['actual']-df['forecast'])
        df[z2] = df[z_]*2
        Z = df[z_].sum() / np.sqrt(df[z2].sum())
        p_value = 1 - stats.norm.cdf(Z)
        
        t = tabulate([[self.alpha,Z,p_value]],headers = ['Alpha','Value of Z','p-Value'])
        print(t)



In [146]:
backES_fhs_99 = Backtest_2(actual = backtest_fhs['daily linearised loss'], forecast = backtest_fhs['ES FHS 99%'], alpha = 0.99)
backES_fhs_95 = Backtest_2(actual = backtest_fhs['daily linearised loss'], forecast = backtest_fhs['ES FHS 95%'], alpha = 0.95)

print('FHS')
backES_fhs_99.test_ES()

backES_fhs_95.test_ES()

FHS
  Alpha    Value of Z    p-Value
-------  ------------  ---------
   0.99      0.254946   0.399382
  Alpha    Value of Z    p-Value
-------  ------------  ---------
   0.95      0.508448    0.30557


In [148]:
backES_hs_99 = Backtest_2(actual = backtest_hs['daily linearised loss'], forecast = backtest_hs['ES 99%'], alpha = 0.99)
backES_hs_95 = Backtest_2(actual = backtest_hs['daily linearised loss'], forecast = backtest_hs['ES 95%'], alpha = 0.95)

print('HS')
backES_hs_99.test_ES()

backES_hs_95.test_ES()

HS
  Alpha    Value of Z    p-Value
-------  ------------  ---------
   0.99      0.341815   0.366245
  Alpha    Value of Z    p-Value
-------  ------------  ---------
   0.95       0.66797   0.252076


In [151]:
backES_garch_99 = Backtest_2(actual = backtest_garch['daily linearised loss'], forecast = backtest_garch['ES GARCH 99%'], alpha = 0.99)
backES_garch_95 = Backtest_2(actual = backtest_garch['daily linearised loss'], forecast = backtest_garch['ES GARCH 95%'], alpha = 0.95)

print('GARCH')
backES_garch_99.test_ES()

backES_garch_95.test_ES()

GARCH
  Alpha    Value of Z    p-Value
-------  ------------  ---------
   0.99      0.222502   0.411962
  Alpha    Value of Z    p-Value
-------  ------------  ---------
   0.95      0.470835   0.318879
